In [19]:
import polars as pl
import numpy as np
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from tqdm.auto import tqdm
import spacy
nlp = spacy.load("/Users/dorukyurdusen/Desktop/Teknofest/ner/training/new_only/model-best", disable=["parser", "tagger", "lemmatizer"])


columns_to_read = ["city_name", "district_name", "quarter_name", "street_name"]
column_types = {
    "city_name": pl.Categorical,
    "district_name": pl.Categorical,
    "quarter_name": pl.Categorical
}
base_df = pl.read_csv(
    
    "/Users/dorukyurdusen/Desktop/Teknofest/dataset/base_df_filtered.csv",
    columns=columns_to_read,  # Sadece bu sütunları diskten oku (ÇOK VERİMLİ)
    schema_overrides=column_types      # Okunan sütunların tiplerini anında ayarla
)
ALL_CITIES = set(base_df.get_column("city_name").drop_nulls().unique().to_list())
ALL_DISTRICTS = set(base_df.get_column("district_name").drop_nulls().unique().to_list())
ALL_QUARTERS = set(base_df.get_column("quarter_name").drop_nulls().unique().to_list())
ALL_STREETS = set(base_df.get_column("street_name").drop_nulls().unique().to_list())


In [20]:


print("Faz 1: Tek Seferlik Kurulum (IndexIVFPQ ile) Başladı...")

# --- Adım 1: Veri Yükleme ve Vektörleştirme (Aynı) ---
# all_streets_list'in hazır olduğunu varsayıyoruz.
all_streets_list = sorted(list(ALL_STREETS))
print(f"{len(all_streets_list)} adet benzersiz sokak adı bulundu.")

vectorizer = TfidfVectorizer(
    analyzer='char',
    ngram_range=(2, 4),
    max_features=10000 
)
vectorizer.fit(all_streets_list)
dimension = len(vectorizer.get_feature_names_out())
print(f"Vektör boyutu: {dimension}")

# --- Adım 2: YENİ FAISS İndeksini Tanımlama ---
# IVF için hücre (klasör) sayısı. Genel kural: Toplam vektör sayısının karekökünün 4 katı.
nlist = int(4 * np.sqrt(len(all_streets_list))) 
# PQ için alt-vektör sayısı. Boyut (dimension) bu sayıya tam bölünmeli. 16, 32, 64 gibi 2'nin katları idealdir.
m = 16
# Her alt-vektör için kullanılacak bit sayısı. 8 bit standarttır (2^8=256 kod).
nbits = 8 

# Önce hücreleri belirlemek için bir "quantizer" (niceleyici) oluşturulur.
quantizer = faiss.IndexFlatL2(dimension)

# Ana IVFPQ indeksini bu quantizer'ı kullanarak oluştur.
index = faiss.IndexIVFPQ(quantizer, dimension, nlist, m, nbits)
print(f"FAISS IndexIVFPQ oluşturuldu. Hücre sayısı (nlist): {nlist}, Sıkıştırma (m): {m}")

# --- Adım 3: İndeksi EĞİTME ---
# Bu en önemli fark! IVFPQ indeksinin, verinin genel dağılımını öğrenmesi gerekir.
# Bunun için tüm veriyi değil, sadece bir örneklemi kullanmak yeterlidir.
print("İndeks, verinin bir örneklemi üzerinde eğitiliyor...")
# Vektörlerin bir kısmını (örn: 100k) alıp yoğun matrise çevirerek eğitelim.
# RAM yetmezse, bu sample_size'ı veya aşağıdaki batch_size'ı düşür.
sample_size = min(100000, len(all_streets_list))
random_sample_indices = np.random.choice(len(all_streets_list), sample_size, replace=False)
sample_streets = [all_streets_list[i] for i in random_sample_indices]

vectors_for_training = vectorizer.transform(sample_streets).toarray().astype(np.float32)
index.train(vectors_for_training)
print("İndeks eğitildi.")

# --- Adım 4: Tüm Veriyi Parçalar Halinde İndekse Ekleme (Aynı) ---
batch_size = 10000
for i in tqdm(range(0, len(all_streets_list), batch_size), desc="Tüm Sokaklar İndekse Ekleniyor"):
    batch_streets = all_streets_list[i : i + batch_size]
    batch_vectors_sparse = vectorizer.transform(batch_streets)
    batch_vectors_dense = batch_vectors_sparse.toarray().astype(np.float32)
    index.add(batch_vectors_dense)

print(f"İndekse {index.ntotal} adet vektör eklendi.")




Faz 1: Tek Seferlik Kurulum (IndexIVFPQ ile) Başladı...
71809 adet benzersiz sokak adı bulundu.
Vektör boyutu: 10000
FAISS IndexIVFPQ oluşturuldu. Hücre sayısı (nlist): 1071, Sıkıştırma (m): 16
İndeks, verinin bir örneklemi üzerinde eğitiliyor...
İndeks eğitildi.


Tüm Sokaklar İndekse Ekleniyor:   0%|          | 0/8 [00:00<?, ?it/s]

İndekse 71809 adet vektör eklendi.


In [ ]:
# --- Adım 5: Varlıkların Diske Kaydedilmesi ---

# Kaydedilecek dosya yollarını tanımla
vectorizer_path = "street_vectorizer.pkl"
index_path = "street_index_ivfpq.faiss"
street_list_path = "all_streets_list.pkl"

# Vektörleştiriciyi pickle ile kaydet
with open(vectorizer_path, 'wb') as f:
    pickle.dump(vectorizer, f)

# Sokak listesini pickle ile kaydet
with open(street_list_path, 'wb') as f:
    pickle.dump(all_streets_list, f)

# FAISS indeksini kendi formatında kaydet (bunu zaten yapmıştın)
faiss.write_index(index, index_path)

print(f"\nKurulum tamamlandı! 3 varlık da diske kaydedildi:")
print(f"- Vektörleştirici: {vectorizer_path}")
print(f"- FAISS İndeksi: {index_path}")
print(f"- Sokak Listesi: {street_list_path}")


Kurulum tamamlandı! 3 varlık da diske kaydedildi:
- Vektörleştirici: street_vectorizer.pkl
- FAISS İndeksi: street_index_ivfpq.faiss
- Sokak Listesi: all_streets_list.pkl


In [23]:
faiss.write_index(index,"street_index_ivfpq.faiss")

In [24]:
import faiss
import pickle
import numpy as np
from rapidfuzz import process, fuzz

# --- Gerekli Arama Varlıklarını Belleğe Yükle ---

# Kaydettiğimiz dosya yolları
VECTORIZER_PATH = "street_vectorizer.pkl"
INDEX_PATH = "street_index_ivfpq.faiss"
STREET_LIST_PATH = "all_streets_list.pkl"

print("Kaydedilmiş arama varlıkları yükleniyor...")
try:
    with open(VECTORIZER_PATH, 'rb') as f:
        vectorizer = pickle.load(f)

    with open(STREET_LIST_PATH, 'rb') as f:
        all_streets_list = pickle.load(f)

    

    index = faiss.read_index(INDEX_PATH)
    print("Varlıklar başarıyla yüklendi.")
    print(f"İndeks Tipi: {type(index)}")
    print(f"İndeksteki Toplam Sokak Sayısı: {index.ntotal}")

except FileNotFoundError:
    print("HATA: Gerekli arama dosyaları bulunamadı. Lütfen önce Faz 1 kurulum script'ini çalıştırın.")
    # Script'i burada durdurmak iyi bir fikir olabilir.
    exit()


# --- Arama Performans Ayarı: nprobe ---
# Bu, FAISS'in arama yaparken kaç tane "klasöre" (centroid'e) bakacağını belirler.
# Değer ne kadar yüksekse, arama o kadar hassas ama bir o kadar yavaş olur.
# Değer ne kadar düşükse, arama o kadar hızlı ama çok nadir de olsa en iyi sonucu kaçırma riski olur.
# Genellikle nlist'in %1'i civarında bir değer iyi bir başlangıçtır.
index.nprobe = 20
print(f"FAISS arama hassasiyeti (nprobe) {index.nprobe} olarak ayarlandı.")

Kaydedilmiş arama varlıkları yükleniyor...
Varlıklar başarıyla yüklendi.
İndeks Tipi: <class 'faiss.swigfaiss_avx2.IndexIVFPQ'>
İndeksteki Toplam Sokak Sayısı: 71809
FAISS arama hassasiyeti (nprobe) 20 olarak ayarlandı.


In [25]:
from rapidfuzz import process,fuzz

In [26]:
turkish_map = str.maketrans("ğüşöçıİ", "gusocii")
def normalize(text: str) -> str:
    if not text:
        return ""
    return str(text).lower().translate(turkish_map).replace("i̇","i").strip()

import re

# Bu desen, bir metnin sonundaki adres anahtar kelimelerini hedefler.
# Sadece kelimeleri değil, başındaki boşluğu ve sonundaki ekleri de ('NDA, 'NE vb.) yakalar.
name_extractor_pattern = re.compile(
    r"\s+\b("  # Anahtar kelimeden önceki boşluğu yakala
    r"MAHALLESİ|MAHALLE|MAH|"
    r"CADDESİ|CADDE|CAD|CD|"
    r"SOKAĞI|SOKAK|SOK|SK|"
    r"BULVARI|BULVAR|BLV|"
    r"MEYDANI|MEYDAN|MEYD|"
    r"APARTMANI|APT|AP|"
    r"SİTESİ|SİTE|SİT"
    r")\b\.?(?:'[A-ZİÖÜÇŞĞ]+)?\s*$",  # Sonundaki olası nokta, 'NDA gibi ekler ve boşlukları yakala
    flags=re.IGNORECASE | re.UNICODE
)

def extract_name_part(text):
    """
    Bir adres bileşeninin sonundaki anahtar kelimeleri silerek
    sadece özel ismi döndürür.
    """
    if not isinstance(text, str):
        return text
        
    # Deseni metinle eşleştir ve eşleşen kısmı boşlukla değiştir.
    cleaned_text = name_extractor_pattern.sub("", text)
    
    # Sonuçta kalabilecek baş/son boşluklarını temizle.
    return cleaned_text.strip()

In [28]:
query = "Yeni Bağarası"
query = normalize(extract_name_part(query))
process.extract(query,ALL_QUARTERS,scorer=fuzz.ratio)

[('yenibagarasi', 96.0, 1008),
 ('yenibaglar', 78.26086956521739, 507),
 ('bagarasi', 76.19047619047619, 1244),
 ('yenibagyaka', 75.0, 2440),
 ('baglarbasi', 69.56521739130434, 916)]

In [87]:

def compare_scorers(query_string: str, choices: list, limit: int = 5):
    """
    Verilen bir sorgu için tüm önemli rapidfuzz scorer'larını dener
    ve sonuçları karşılaştırmalı bir Polars DataFrame'i olarak döndürür.
    """
    # 1. Sorguyu hazırla
    processed_query = normalize(extract_name_part(query_string))
    
    # 2. Test edilecek scorer'ların listesi
    scorers_to_test = {
        "ratio": fuzz.ratio,
        #"partial_ratio": fuzz.partial_ratio,
        "token_sort_ratio": fuzz.token_sort_ratio,
        "token_set_ratio": fuzz.token_set_ratio,
        "partial_token_set_ratio": fuzz.partial_token_set_ratio, # Tavsiyem
        "WRatio": fuzz.WRatio, # Varsayılan
        "QRatio": fuzz.QRatio
    }
    
    all_results = []
    
    print(f"Sorgu: '{processed_query}'")
    
    # 3. Her bir scorer için döngüye gir ve en iyi sonuçları bul
    for name, scorer_func in scorers_to_test.items():
        # process.extract ile en iyi 'limit' adet sonucu al
        best_matches = process.extract(
            processed_query,
            choices,
            scorer=scorer_func,
            limit=limit
        )
        
        # Sonuçları daha sonra DataFrame'e çevirmek üzere formatla
        for match, score, index in best_matches:
            all_results.append({
                "scorer": name,
                "match": match,
                "score": score
            })
            
    # 4. Tüm sonuçları tek bir Polars DataFrame'ine dönüştür
    if not all_results:
        return pl.DataFrame({"scorer": [], "match": [], "score": []})
        
    return pl.DataFrame(all_results)

In [10]:
extract_name_part(nlp("Yeni Bağarası mahallesi yeni Foça caddesi no 22/1 1").ents[0].text)

'Yeni Bağarası'

In [11]:
compare_df = compare_scorers("yeni bagarasi",ALL_QUARTERS)

NameError: name 'compare_scorers' is not defined

In [29]:
query = "Mehmet Kocatepe caddesi"
query = extract_name_part(query)
query = normalize(query)
query_vector = vectorizer.transform([query]).toarray().astype(np.float32)
distances, indices = index.search(query_vector, 10)
candidate_streets = [all_streets_list[i] for i in indices[0]]
print(candidate_streets)
process.extractOne(query,candidate_streets,scorer = fuzz.token_set_ratio)



['mehmet kocar', 'mehmet kosar', 'mehmet kocatepe (krc)', 'mehmet kocar/1', 'mehmet korucu', 'kocatepe', 'mehmet kurt', 'mehmet kiyak', 'mehmet kaya', 'kocatepe  4.']


('mehmet kocatepe (krc)', 100.0, 2)

In [30]:
query = "Mehmet Kocatepe caddesi"
query = extract_name_part(query)
query = normalize(query)
query_vector = vectorizer.transform([query]).toarray().astype(np.float32)

In [15]:
distances, indices = index.search(query_vector, 10)
indices

array([[   12,    13,    16,  1418,  1971, 51226, 51617, 72420, 72424,
        72425]])

In [21]:
len(all_streets_list)

71809